In [35]:
import os
import struct
import numpy as np

"""
Loosely inspired by http://abel.ee.ucla.edu/cvxopt/_downloads/mnist.py
which is GPL licensed.
"""

def read(dataset = "training", path = "."):
    """
    Python function for importing the MNIST data set.  It returns an iterator
    of 2-tuples with the first element being the label and the second element
    being a numpy.uint8 2D array of pixel data for the given image.
    """

    if dataset is "testing":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset is "training":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise Exception("dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

    return img, lbl

In [36]:
# MNIST data directory path
path = ''

In [37]:
X_train, y_train = read("training", path)
X_test, y_test = read("testing", path)

In [38]:
# reshape matrix to vector
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

In [39]:
from sklearn.preprocessing import scale

# Normalization per image
X_train = scale(X_train, axis=1)
X_test = scale(X_test, axis=1)

In [40]:
#random_state = 108

In [41]:
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

# hyper parameter candidate
penalty = ['l2']
C       = [1e-5, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]
solvers  = ['sag', 'saga']

In [42]:
df = pd.DataFrame(columns=['solver', 'penalty', 'C value', 'f1 score', 'learning time'])
bookmark = 0

for solver in solvers:
    for p in penalty:
        for c in C:
            try:
                
                logisticRegr = LogisticRegression(penalty=p, C=c, solver = solver,
                                                  multi_class='multinomial', random_state=108)
                begin_time = datetime.now()
                logisticRegr.fit(X_train,y_train)
                              
                end_time = datetime.now()
                
                y_pred = logisticRegr.predict(X_test)
                score = f1_score(y_test, y_pred, average='macro')
                df.loc[bookmark] = [solver, p, c, score, end_time - begin_time]
                print(df.loc[bookmark])
                df.to_csv("LR_ovr.csv", mode='w')

                bookmark = bookmark + 1                
            except Exception as ex:
                print('ERROR:', ex)
                break
            finally:
                print('=======================================')

df.to_csv("LR_multi.csv", mode='w')

solver                              sag
penalty                              l2
C value                           1e-05
f1 score                       0.765129
learning time    0 days 00:00:03.890246
Name: 0, dtype: object
solver                              sag
penalty                              l2
C value                          0.0001
f1 score                       0.859847
learning time    0 days 00:00:08.300043
Name: 1, dtype: object
solver                              sag
penalty                              l2
C value                           0.001
f1 score                       0.895419
learning time    0 days 00:00:31.115744
Name: 2, dtype: object
solver                              sag
penalty                              l2
C value                            0.01
f1 score                       0.906052
learning time    0 days 00:00:37.085825
Name: 3, dtype: object
solver                              sag
penalty                              l2
C value                     